# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [16]:
import pandas as pd

def cargar_datos():
    colspecs = [(0, 10),   
                (10, 40),  
                (40, 50),  
                (50, 55)]
    
    df = pd.read_fwf('datos.dat', colspecs=colspecs, header=None,
                    names=['fecha', 'producto', 'precio', 'cantidad'])
    
    df['precio'] = pd.to_numeric(df['precio'], errors='coerce')
    df['cantidad'] = pd.to_numeric(df['cantidad'], errors='coerce')
    
    return df

# Cargar los datos desde el archivo
datos_df = cargar_datos()

# Mostrar los primeros 5 registros
print(datos_df.head())

# Mostrar todos los registros
# print(datos_df)


        fecha    producto  precio  cantidad
0  2024-08-27     Mirinda    1510        14
1  2024-08-27     Mirinda    1560        12
2  2024-08-28     Torasso     940         8
3  2024-08-29  Pepsi Cola    1210        10
4  2024-08-30     Mirinda    1520         1


### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [17]:
def calcular_totales(df):
    # Calcular el total de importe (precio * cantidad)
    total_importe = (df['precio'] * df['cantidad']).sum()
    
    # Calcular el total de cantidad de productos vendidos
    total_cantidad = df['cantidad'].sum()
    
    return total_importe, total_cantidad

importe, cantidad = calcular_totales(datos_df)

print(f"Las ventas fueron de ${importe:.2f} en {cantidad} unidades")


Las ventas fueron de $392730.00 en 335 unidades


### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [18]:
def unidades_vendidas(df):
    # Agrupar por producto y sumar la cantidad de cada producto
    ventas_por_producto = df.groupby('producto')['cantidad'].sum()
    
    return ventas_por_producto

def listar_ventas(ventas):
    print("Unidades vendidas por producto:")
    for producto, cantidad in ventas.items():
        print(f"• {producto}: {cantidad} unidades")

ventas = unidades_vendidas(datos_df)
listar_ventas(ventas)


Unidades vendidas por producto:
• Coca Cola: 57 unidades
• Mirinda: 85 unidades
• Pepsi Cola: 89 unidades
• Sprite: 72 unidades
• Torasso: 32 unidades


###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [19]:
def precio_promedio(df):
    # Agrupar por producto y calcular el promedio de precios
    promedio_por_producto = df.groupby('producto')['precio'].mean()
    
    return promedio_por_producto

def listar_precios(precios):
    print("Precio promedio por producto:")
    for producto, promedio in precios.items():
        print(f"• {producto}: ${promedio:.2f}")

precios = precio_promedio(datos_df)
listar_precios(precios)

Precio promedio por producto:
• Coca Cola: $1072.50
• Mirinda: $1545.83
• Pepsi Cola: $1245.00
• Sprite: $841.43
• Torasso: $920.00


### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [20]:
def ranking_productos(df, top=3):
    ventas_por_producto = df.groupby('producto')['cantidad'].sum()

    # Ordenar por la cantidad de unidades vendidas en orden descendente y seleccionar los top productos
    ranking = ventas_por_producto.sort_values(ascending=False).head(top)  
    
    return ranking

def listar_ranking(ranking):
    print("Ranking de productos más vendidos:")
    for i, (producto, cantidad) in enumerate(ranking.items(), 1):
        print(f"{i}. {producto}: {cantidad} unidades")

ranking = ranking_productos(datos_df)
listar_ranking(ranking)

Ranking de productos más vendidos:
1. Pepsi Cola: 89 unidades
2. Mirinda: 85 unidades
3. Sprite: 72 unidades


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [21]:
def ventas_por_mes(df):
    # Extraer el año y mes directamente como una cadena (YYYY-MM)
    df['mes'] = df['fecha'].str[:7]  # Extrae los primeros 7 caracteres de la fecha
    
    # Agrupar por mes y producto, luego sumar las cantidades
    ventas_mensuales = df.groupby(['mes', 'producto'])['cantidad'].sum().reset_index()
    
    return ventas_mensuales

def listar_ventas_mensuales(ventas):
    for _, row in ventas.iterrows():
        print(f"Mes: {row['mes']}, Producto: {row['producto']}, Unidades vendidas: {row['cantidad']}")

ventas = ventas_por_mes(datos_df)
listar_ventas_mensuales(ventas)

Mes: 2024-08, Producto: Mirinda, Unidades vendidas: 27
Mes: 2024-08, Producto: Pepsi Cola, Unidades vendidas: 10
Mes: 2024-08, Producto: Torasso, Unidades vendidas: 8
Mes: 2024-09, Producto: Coca Cola, Unidades vendidas: 57
Mes: 2024-09, Producto: Mirinda, Unidades vendidas: 58
Mes: 2024-09, Producto: Pepsi Cola, Unidades vendidas: 79
Mes: 2024-09, Producto: Sprite, Unidades vendidas: 72
Mes: 2024-09, Producto: Torasso, Unidades vendidas: 24


### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [22]:
def resumen_ventas(df):
    # Calcular las métricas necesarias por producto
    resumen = df.groupby('producto').agg(
        precio_promedio=('precio', 'mean'),
        total_unidades=('cantidad', 'sum'),
        importe_total=('precio', lambda x: (x * df.loc[x.index, 'cantidad']).sum())
    ).reset_index()
    
    # Ordenar por el nombre del producto alfabéticamente
    resumen = resumen.sort_values(by='producto')
    
    return resumen

def informe_ventas(resumen):
    # Encabezado del informe
    print(f"{'Producto':<30} {'Precio Promedio':>15} {'Unidades Vendidas':>20} {'Importe Total':>15}")
    print("="*83)
    
    # Mostrar la información de cada producto
    for _, row in resumen.iterrows():
        print(f"{row['producto']:<30} {row['precio_promedio']:>15.2f} {row['total_unidades']:>20} {row['importe_total']:>15.2f}")

resumen = resumen_ventas(datos_df)
informe_ventas(resumen)

Producto                       Precio Promedio    Unidades Vendidas   Importe Total
Coca Cola                              1072.50                   57        60780.00
Mirinda                                1545.83                   85       131080.00
Pepsi Cola                             1245.00                   89       110510.00
Sprite                                  841.43                   72        61040.00
Torasso                                 920.00                   32        29320.00


## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [23]:
import numpy as np
def f(x, coeficientes):
    a,b,c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return y - y_pred

X = np.array([0,1,2,3,5])
Y = np.array([0,8,12,12,0])

def buscar_coeficientes():
    # Ajustar un polinomio de grado 2 (parámetro deg=2) a los puntos X, Y
    coeficientes = np.polyfit(X, Y, deg=2)
    return coeficientes

coeficientes = buscar_coeficientes()
coeficientes

array([-2.00000000e+00,  1.00000000e+01, -1.23812445e-14])

### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [24]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a,b,c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return np.sum((y - y_pred)**2)

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8,12,11, 1]) # Observar que no son los mismos valores que en el ejemplo anterior

def buscar_coeficientes(X, Y, aprendizaje=0.001, error_tolerancia=1):
    # Generar coeficientes aleatorios iniciales entre -10 y 10
    coeficientes = randint(-10, 10, 3)
    
    # Calcular el error inicial
    y_pred = f(X, coeficientes)
    err_actual = error(Y, y_pred)
    
    # Iterar mientras el error sea mayor que la tolerancia
    while err_actual > error_tolerancia:
        # Generar un pequeño valor aleatorio con randint para modificar los coeficientes
        # Este valor puede ser entre -1 y 1, multiplicado por el aprendizaje
        ajuste = (randint(-1, 2, 3)) * aprendizaje  # randint entre -1 y 1
        
        # Actualizar los coeficientes
        nuevos_coeficientes = coeficientes + ajuste
        
        # Calcular el nuevo error con los nuevos coeficientes
        y_pred_nuevo = f(X, nuevos_coeficientes)
        err_nuevo = error(Y, y_pred_nuevo)
        
        # Si el nuevo error es menor, actualizamos los coeficientes
        if err_nuevo < err_actual:
            coeficientes = nuevos_coeficientes
            err_actual = err_nuevo
    
    return coeficientes

coeficientes = buscar_coeficientes(X, Y)
coeficientes

array([-1.823e+00,  9.305e+00, -3.000e-03])